In [88]:
import numpy as np

In [89]:
# Given Parameters
transmission_rate = [500, 1000, 1500, 2000, 2500, 3000]  # in kb/s
num_devices_list = [50, 100, 150, 200, 250, 300]
task_data = [5, 10, 15, 20, 25, 30, 35, 40, 45]  # in Mbit
max_delay = 0.5  # in seconds
device_computing_capacity = [0.5, 1]  # in GHz
num_of_mecs = 10
mec_computing_capacity = 4  # in GHz
transmission_power = 0.5  # in Watts

# Constants
H_m = 1e-96  # Consumption factor of electricity
y_nt = 1e-6  # Weighted factor of local time cost
y_ne = 1e-12  # Weighted factor of local energy consumption
e_m = 1e-15  # Energy required to calculate a single bit of task data for MEC server
noise_power_spectral_density = 1e-9  # in Watts/Hz
channel_gain = 2e-10  # 
path_loss_index = 4  # Typical urban area path loss exponent
bandwidth = 1e9  # 1 MHz bandwidth

# Assumed Parameters for Data Compression
compression_ratio = 2  # Example compression ratio
E_comp = 1e-9  # Energy for compression per bit (in Joules)
E_decomp = 1e-9  # Energy for decompression per bit (in Joules)
T_comp = 1e-6  # Time for compression per bit (in seconds)
T_decomp = 1e-6  # Time for decompression per bit (in seconds

In [90]:
def fitness_function(x, task_data, transmission_power, bandwidth, noise_power_spectral_density, device_computing_capacity, mec_computing_capacity):
    compressed_data = task_data / compression_ratio  # Data size after compression
    
    if x == 0:  # Local computation (compression not beneficial for local computation)
        local_time_cost = task_data / (device_computing_capacity * 1e9)  # Converting GHz to Hz
        local_energy_cost = H_m * (device_computing_capacity**2) * task_data
        total_cost = y_nt * local_time_cost + y_ne * local_energy_cost
    else:  # Edge server computation with data compression
        compression_time = T_comp * task_data
        decompression_time = T_decomp * compressed_data
        transmission_time = compressed_data / (bandwidth * np.log2(1 + (transmission_power * channel_gain) / (noise_power_spectral_density * bandwidth)))
        mec_task_execution_time = compressed_data / (mec_computing_capacity * 1e9)  # Converting GHz to Hz
        
        total_time_cost = compression_time + transmission_time + decompression_time + mec_task_execution_time
        compression_energy_cost = E_comp * task_data
        decompression_energy_cost = E_decomp * compressed_data
        mec_energy_cost = transmission_power * transmission_time + e_m * compressed_data
        
        total_cost = y_nt * total_time_cost + y_ne * (compression_energy_cost + decompression_energy_cost + mec_energy_cost)

    return total_cost


In [91]:
from concurrent.futures import ThreadPoolExecutor, as_completed

class QPSO:
    def __init__(self, num_devices, max_iter, task_data):
        self.num_devices = num_devices
        self.max_iter = max_iter
        self.task_data = np.array(task_data) * 1e6  # Convert Mbit to bits for task sizes
        self.X = np.random.randint(2, size=(num_devices, max_iter))  # Initial binary decisions for each device
        
        # Initialize Personal best decisions and their fitness
        self.P = np.zeros((num_devices, 1), dtype=int)
        self.P_fitness = np.full(num_devices, np.inf)

        # Initialize global best decision and its fitness
        self.g = np.zeros(num_devices, dtype=int)  # Placeholder, will be updated in evaluate_initial_decisions_parallel
        self.g_fitness = np.inf  # Important: initialize g_fitness before evaluate_initial_decisions_parallel

        # Parallel evaluation of initial decisions to find personal bests
        self.evaluate_initial_decisions_parallel()

        # Initialize global best decision set
        self.g = self.X[:, 0]
        self.g_fitness = np.min(self.P_fitness)  # Use the best of the initial personal bests
        g_index = np.argmin(self.P_fitness)
        self.g = self.X[:, g_index]

    def evaluate_fitness_parallel(self, tasks):
        with ThreadPoolExecutor() as executor:
            # Create a mapping of future to (i, t) pair
            future_to_it = {}
            
            for i, (task_size, device_capacity, t) in enumerate(tasks):
                future = executor.submit(fitness_function, self.X[i, t], task_size, transmission_power,
                                        bandwidth, noise_power_spectral_density, device_capacity, mec_computing_capacity)
                future_to_it[future] = (i, t)

            for future in as_completed(future_to_it):
                i, t = future_to_it[future]
                current_fitness = future.result()
                # Update personal and potentially global best
                if current_fitness < self.P_fitness[i]:
                    self.P[i, 0] = self.X[i, t]
                    self.P_fitness[i] = current_fitness
                    if current_fitness < self.g_fitness:
                        self.g = self.X[:, t]
                        self.g_fitness = current_fitness


    def evaluate_initial_decisions_parallel(self):
        tasks = [(self.task_data[i % len(self.task_data)], device_computing_capacity[i % len(device_computing_capacity)], 0) for i in range(self.num_devices)]
        self.evaluate_fitness_parallel(tasks)

    def run(self):
        for t in range(self.max_iter):
            tasks = [(self.task_data[i % len(self.task_data)], device_computing_capacity[i % len(device_computing_capacity)], t) for i in range(self.num_devices)]
            self.evaluate_fitness_parallel(tasks)

            # Quantum-inspired position updates for next iteration
            for i in range(self.num_devices):
                if t < self.max_iter - 1:
                    phi = np.random.uniform(0, 1)
                    self.X[i, t+1] = np.round(phi * self.P[i, 0] + (1 - phi) * self.g[i]).astype(int)
        return self.g

In [92]:
# Collect the best offloading strategies for each number of devices
best_offloading_strategies = {}
for num_devices in num_devices_list:
    qpso = QPSO(num_devices, max_iter=100, task_data=task_data)
    best_offloading_strategies[num_devices] = qpso.run()

In [93]:
#print decisions
best_offloading_strategies

{50: array([1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1]),
 100: array([0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
        1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1,
        1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
 150: array([1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1,
        1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
        0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0,
        0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0,
        0, 0, 1, 1, 0, 1

In [96]:
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

# Constants and Given Parameters
transmission_rate = [500, 1000, 1500, 2000, 2500, 3000]  # in kb/s
num_devices_list = [50, 100, 150, 200, 250, 300]
task_data = [5, 10, 15, 20, 25, 30, 35, 40, 45]  # in Mbit
device_computing_capacity = [0.5, 1]  # in GHz
mec_computing_capacity = 4  # in GHz
transmission_power = 0.5  # in Watts

H_m = 1e-96  # Electricity consumption factor
y_nt = 1e-6  # Weighted factor of local time cost
y_ne = 1e-12  # Weighted factor of local energy consumption
e_m = 1e-15  # Energy required to calculate a single bit for MEC server
noise_power_spectral_density = 1e-9  # in Watts/Hz
channel_gain = 2e-10  
bandwidth = 1e9  # 1 GHz bandwidth

def fitness_function(x, task_data, transmission_power, bandwidth, noise_power_spectral_density, device_computing_capacity, mec_computing_capacity, transmission_rate):
    task_data_bits = task_data * 1e6  # Convert Mbit to bits
    if x == 0:  # Local computation
        local_time_cost = task_data_bits / (device_computing_capacity * 1e9)  # GHz to Hz
        local_energy_cost = H_m * (device_computing_capacity**2) * task_data_bits
        total_cost = y_nt * local_time_cost + y_ne * local_energy_cost
    else:  # Edge server computation
        transmission_time = task_data_bits / (transmission_rate * 1e3)  # Convert kb/s to b/s
        mec_task_execution_time = task_data_bits / (mec_computing_capacity * 1e9)
        total_time_cost = transmission_time + mec_task_execution_time
        mec_energy_cost = transmission_power * transmission_time + e_m * task_data_bits
        total_cost = y_nt * total_time_cost + y_ne * mec_energy_cost
    return total_cost

class QPSO:
    def __init__(self, num_devices, max_iter, task_data, transmission_rate):
        self.num_devices = num_devices
        self.max_iter = max_iter
        self.task_data = np.array(task_data)  # Mbit, no conversion here
        self.transmission_rate = transmission_rate  # New
        self.X = np.random.randint(2, size=(num_devices, max_iter))
        self.P = np.zeros((num_devices, 1), dtype=int)
        self.P_fitness = np.full(num_devices, np.inf)
        self.g = np.zeros(num_devices, dtype=int)
        self.g_fitness = np.inf
        self.evaluate_initial_decisions_parallel()

    def evaluate_fitness_parallel(self, tasks):
        with ThreadPoolExecutor() as executor:
            future_to_it = {}
            for i, task in enumerate(tasks):
                future = executor.submit(fitness_function, self.X[i, task[2]], task[0], transmission_power,
                                         bandwidth, noise_power_spectral_density, task[1], mec_computing_capacity,
                                         self.transmission_rate)
                future_to_it[future] = i
            for future in as_completed(future_to_it):
                i = future_to_it[future]
                current_fitness = future.result()
                if current_fitness < self.P_fitness[i]:
                    self.P[i] = self.X[i, tasks[i][2]]
                    self.P_fitness[i] = current_fitness
                    if current_fitness < self.g_fitness:
                        self.g = self.X[:, tasks[i][2]]
                        self.g_fitness = current_fitness

    def evaluate_initial_decisions_parallel(self):
        tasks = [(self.task_data[i % len(self.task_data)], device_computing_capacity[i % len(device_computing_capacity)], 0) for i in range(self.num_devices)]
        self.evaluate_fitness_parallel(tasks)

    def run(self):
        for t in range(1, self.max_iter):
            tasks = [(self.task_data[i % len(self.task_data)], device_computing_capacity[i % len(device_computing_capacity)], t) for i in range (self.num_devices)]
            self.evaluate_fitness_parallel(tasks)
            # Quantum-inspired position updates for next iteration
            for i in range(self.num_devices):
                if t < self.max_iter - 1:
                    phi = np.random.uniform(0, 1)
                    self.X[i, t+1] = np.round(phi * self.P[i] + (1 - phi) * self.g[i]).astype(int)
        return self.g, self.g_fitness

def main(transmission_rates, num_devices_list, task_data, device_computing_capacity):
    overall_energy_consumption_results = {}
    for rate in transmission_rates:
        rate_results = {}
        for num_devices in num_devices_list:
            qpso = QPSO(num_devices, max_iter=100, task_data=task_data, transmission_rate=rate)
            _, best_fitness = qpso.run()  # best_fitness here is interpreted as the minimal energy consumption
            rate_results[num_devices] = best_fitness
        overall_energy_consumption_results[rate] = rate_results
    return overall_energy_consumption_results

# Execute the main function to get energy consumption results
energy_consumption_results = main(transmission_rate, num_devices_list, task_data, device_computing_capacity)

# Print the energy consumption results
for rate, results in energy_consumption_results.items():
    print(f"Transmission Rate: {rate} kb/s")
    for num_devices, energy_consumption in results.items():
        print(f"  Devices: {num_devices}, Energy Consumption: {energy_consumption:.2e} J")


Transmission Rate: 500 kb/s
  Devices: 50, Energy Consumption: 5.00e-09 J
  Devices: 100, Energy Consumption: 5.00e-09 J
  Devices: 150, Energy Consumption: 5.00e-09 J
  Devices: 200, Energy Consumption: 5.00e-09 J
  Devices: 250, Energy Consumption: 5.00e-09 J
  Devices: 300, Energy Consumption: 5.00e-09 J
Transmission Rate: 1000 kb/s
  Devices: 50, Energy Consumption: 5.00e-09 J
  Devices: 100, Energy Consumption: 5.00e-09 J
  Devices: 150, Energy Consumption: 5.00e-09 J
  Devices: 200, Energy Consumption: 5.00e-09 J
  Devices: 250, Energy Consumption: 5.00e-09 J
  Devices: 300, Energy Consumption: 5.00e-09 J
Transmission Rate: 1500 kb/s
  Devices: 50, Energy Consumption: 5.00e-09 J
  Devices: 100, Energy Consumption: 5.00e-09 J
  Devices: 150, Energy Consumption: 5.00e-09 J
  Devices: 200, Energy Consumption: 5.00e-09 J
  Devices: 250, Energy Consumption: 5.00e-09 J
  Devices: 300, Energy Consumption: 5.00e-09 J
Transmission Rate: 2000 kb/s
  Devices: 50, Energy Consumption: 5.00e-0